In [1]:
import sys

if str(type(sys.stdout)) == "<class 'ipykernel.iostream.OutStream'>":
    saved_std = sys.stdout
else:
    sys.stdout = saved_std

sys.stdout

In [2]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Binarizer
from category_encoders.binary import BinaryEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              AdaBoostClassifier,ExtraTreesClassifier, VotingClassifier)
from xgboost import XGBClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string

In [3]:
df = pd.read_csv('train.csv', index_col='PassengerId')
test = pd.read_csv('test.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


Let us note that only people with a Pclass of 1 have a cabin.

In [4]:
# OHE: ['Sex','Title','Pclass','Embarked','Cabin']
df['Surname'] =  [Name.split(',')[0] for Name in df['Name']]
test['Surname'] =  [Name.split(',')[0] for Name in test['Name']]

In [5]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    print(big_string)
    return None

# Give credit

In [6]:
# Titles = ['Mr','Mrs','Miss','Master'] 

# # all other titles must be found and entered by hand
# # since there is no pattern to their placement
# for Name in df['Name']:
#     if substrings_in_string(Name,Titles) == None:
#         Title = input("Please enter this person's title:")
#         Titles.append(Title)

# for Name in test['Name']:
#     if substrings_in_string(Name,Titles) == None:
#         Title = input("Please enter this person's title:")
#         Titles.append(Title)

Titles = ['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme',
          'Ms', 'Major', 'Mlle', 'Col', 'Capt', 'Countess', 'Jonkheer']


df['Title'] = df['Name'].apply(substrings_in_string, substrings=Titles)
test['Title'] = test['Name'].apply(substrings_in_string, substrings=Titles)

In [7]:
# Now we'd like to categories them:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [8]:
df['Title'] = df.apply(replace_titles, axis=1)
test['Title'] = test.apply(replace_titles, axis=1)

In [9]:
df['Cabin'].fillna('U',inplace=True)
test['Cabin'].fillna('U',inplace=True)
df['Deck'] = [Cab[0] for Cab in df['Cabin']]
test['Deck'] = [Cab[0] for Cab in test['Cabin']]
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Deck
PassengerId,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,Braund,Mr,U
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mr,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,Heikkinen,Miss,U
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mr,C
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,Allen,Mr,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,U,S,Montvila,Mr,U
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Graham,Miss,B
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,U,S,Johnston,Miss,U


In [10]:
Numerical_Pre = make_pipeline(SimpleImputer(strategy='mean'))
OneHot_Pre = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())
Binary_Pre = make_pipeline(SimpleImputer(strategy='most_frequent'),BinaryEncoder())
Col_Num = ['Age','Fare','SibSp','Parch']
Col_OHE = ['Sex','Deck','Title','Embarked','Pclass']
Col_Bin = ['Surname','Ticket']
# I can do this becuase none of them have missing values,
# The Imputer is there merely as a formality

Preprocess = ColumnTransformer([('Col_OHE', OneHot_Pre, Col_OHE),
                                ('Col_Bin', Binary_Pre, Col_Bin),
                                ('Col_Num', Numerical_Pre, Col_Num),
                                ])
y = df['Survived']
df.drop(['Survived'],axis=1,inplace=True)

In [11]:
data = Preprocess.fit_transform(df)
test_data = Preprocess.transform(test)

C:\Users\alon4\anaconda3\envs\pythonProject\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [12]:
# Gradient = GradientBoostingClassifier(warm_start=True)
# params_Grad = dict(n_estimators = [200,300],
#                    max_depth=[3,5], 
#                    random_state=[12],
#                    learning_rate=[0.1,0.15])

Gradient = GradientBoostingClassifier(warm_start=True, n_estimators = 200, 
                                      max_depth=4,random_state=12,learning_rate=0.1)

In [13]:
def print_best(Grid,Prints = True):
    if Prints:
        print(Grid.best_score_,Grid.best_params_)
    if not Prints:
        return [Grid.best_score_,Grid.best_params_]

In [14]:
def fit_verb(Grid,data,y,nbOut = saved_std):
    sys.stdout = open(1, 'w')
    Grid.fit(data,y)
    print('Done with grid :)')
    sys.stdout = nbOut

In [23]:
# Grad_Grid = GridSearchCV(Gradient,param_grid=params_Grad,scoring='accuracy',cv=10,verbose=10)

# fit_verb(Grad_Grid,data,y)
# print_best(Grad_Grid)

print(cross_val_score(Gradient,data,y,cv=5,scoring='accuracy',n_jobs=1).mean())

Gradient.fit(data,y)
print(metrics.confusion_matrix(y,Gradient.predict(data)))

Grad_pred = pd.Series(Gradient.predict(test_data), index=test.index, name='Survived')
Grad_pred.to_csv('Predictions_Feat/Feat_Grad_Pred_2.csv')


0.8350072186303434
[[529  20]
 [ 56 286]]


In [43]:
xgbtree1 = XGBClassifier(eta=0.01,use_label_encoder=False, eval_metric='rmse', objective='binary:logistic')
xgbtree2 = XGBClassifier(booster = 'dart',use_label_encoder=False, eval_metric='rmse', objective='binary:logistic')
xgblin = XGBClassifier(booster = 'gblinear',use_label_encoder=False, eval_metric='rmse', objective='binary:logistic')

print(cross_val_score(xgbtree1,data,y,cv=5,scoring='accuracy',n_jobs=1).mean())
print(cross_val_score(xgbtree2,data,y,cv=5,scoring='accuracy',n_jobs=1).mean())
print(cross_val_score(xgblin,data,y,cv=5,scoring='accuracy',n_jobs=1).mean())

xgbtree1.fit(data,y)
print(metrics.confusion_matrix(y,xgbtree1.predict(data)))

xgb_pred = pd.Series(xgbtree1.predict(test_data), index=test.index, name='Survived')
xgb_pred.to_csv('Predictions_Feat/Feat_xgb_Pred_0.csv')

0.8350134957002073
0.8305818843763731
0.8114744837110036
[[519  30]
 [ 64 278]]
